In [53]:
import os
import yaml
from geopandas import GeoDataFrame
import pytest
from network_wrangler import ProjectCard
import numpy as np
import pandas as pd

## remote i/o

In [54]:
input_dir = "."
card_dir = os.path.join(input_dir, 'projects')

filename_node = os.path.join(input_dir, 'node_registry.csv')
filename_output_node = os.path.join(input_dir, 'test_node_registry_update.csv')
filename_config = os.path.join(input_dir, 'registry_config.yml')

## data reads

In [135]:
card_filenames = []
for (dirpath, dirnames, filenames) in os.walk(card_dir):
    for filename in filenames:
        name, extension = os.path.splitext(filename)
        if (extension in ['.yml', '.yaml']):
            card_filenames.append(os.path.join(card_dir, filename))
    break

card_filenames

['./projects/project_B.yml', './projects/project_A.yml']

In [142]:
node_df = pd.read_csv(filename_node)
with open(filename_config, "r") as config_file:
    config_dict = yaml.safe_load(config_file)

## update the registry

In [143]:
card_index = 0
for card_filename in card_filenames:
    write_updated_card = False
    card = ProjectCard.read(card_filename, validate = False)
    card_dict = card.__dict__

    if (card_dict['category'] == 'New Roadway'):
    
        node_index = 0
        for node in card_dict['nodes']:
            new_node = node['model_node_id']
        
            if (new_node <= config_dict['start_node_number']):
                msg = "New node number ({}) in project '{}' is less than the starting node number in config file of {}".format(
                        new_node, 
                        card_dict['project'], 
                        config_dict['start_node_number'],
                    )
                raise ValueError(msg)
            
            if (new_node not in node_df['node'].values):
                df = pd.DataFrame( { 'node':[new_node], 'project':[card_dict['project']] } ) 
                node_df = node_df.append(df)
            else:
                number = 1 + node_df['node'].max()
                card_dict['nodes'][node_index]['model_node_id'] = number
                for i in range(0, len(card_dict['links'])):
                    if (card_dict['links'][i]['A'] == new_node):
                        card_dict['links'][i]['A'] = number
                    if (card_dict['links'][i]['B'] == new_node):
                        card_dict['links'][i]['B'] = number
                df = pd.DataFrame( { 'node':[number], 'project':[card_dict['project']] } ) 
                node_df = node_df.append(df)
                write_updated_card = True
                
            node_index = node_index + 1

    if (write_updated_card):
        card.__dict__.update(card_dict)
        name, extension = os.path.splitext(card_filename)
        card.write(filename = (name + '_updated' + extension))
    
node_df

2021-01-22 15:36:56, INFO: Wrote project card to: ./projects/project_A_updated.yml


,node,project
0,1001,Project B
0,1002,Project B
0,1003,Project A
0,1004,Project A


## write to disk

In [144]:
node_df.to_csv(filename_output_node)